In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
from datetime import datetime 
import numpy as np
import json

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'

In [ ]:
json_path_Subject = 'Sub007\\Ward'

all_json_files = glob.glob(os.path.join(json_path_project, json_path_Subject, '*.json'))
filtered_files = [file for file in all_json_files if not os.path.basename(file).startswith('MetaTable') and not os.path.basename(file).startswith('Sub')]
filtered_files

In [ ]:
'''with open(os.path.join(json_path, 'Report_Json_Session_Report_20200605T115803.json')) as file:
            # Load the JSON data
            data = json.load(file)
'''

In [ ]:
datetime.strptime('2020-09-11T06:33:32Z', "%Y-%m-%dT%H:%M:%SZ")

In [ ]:
reload(extract_features)
SUBID = 'Sub007'
json_path = os.path.join(json_path_project, json_path_Subject)
extract_features.extract_MetaTable(json_path_Subject, json_path, SUBID, filtered_files)


In [ ]:
filename

In [ ]:
mySub = 'Sub005'
All_MetaTables_ThisSub_path = os.path.join(json_path_project, mySub)

# Initialize an empty list to store the DataFrames
dfs = []

# Iterate through the files in the directory
for filename in os.listdir(All_MetaTables_ThisSub_path):
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(All_MetaTables_ThisSub_path, filename)
        
        # Read the JSON file and append the DataFrame to the list
        df = pd.read_json(file_path)
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
All_MetaTables_ThisSub_Combined = pd.concat(dfs, ignore_index=True)
conversion_factor = 24 * 60 * 60
All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] = All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial'] / conversion_factor
All_MetaTables_ThisSub_Combined.sort_values('AccumulatedTherapyOnTimeSinceImplant_Initial', inplace=True)
All_MetaTables_ThisSub_Combined.head()

In [ ]:
# Get the last row of 'BatteryPercentage' for each unique 'Con_Reason' value
last_battery_percentages = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['BatteryPercentage'].last()

# Calculate the sum of 'overallSensingDurSec' for each unique 'Con_Reason' value
sum_sensing_duration = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['overallSensingDurSec'].sum()/60
sum_telemetry_duration = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['telemetry_durationSec'].sum()/60

# Iterate through unique 'Con_Reason' values
for con_reason in All_MetaTables_ThisSub_Combined['Con_Reason'].unique():
    print(f"Last Battery Percentage for {con_reason}: {last_battery_percentages[con_reason]}")
    print(f"Sum of overallSensingDurSec for {con_reason}: {sum_sensing_duration[con_reason]}")
    print(f"Sum of telemetry_durationSec for {con_reason}: {sum_telemetry_duration[con_reason]}")
    print()

In [ ]:
%matplotlib qt
plt.scatter(All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'], All_MetaTables_ThisSub_Combined['BatteryPercentage'])
plt.show()

In [ ]:
All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] < 50, 'FollowUp'] = 'FU0M'

All_MetaTables_ThisSub_Combined.loc[
    (All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > 50) &
    (All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] < 300), 'FollowUp'] = 'FU3M'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] >300, 'FollowUp'] = 'FU12M'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Beelitz', 'FollowUp'] = 'Beelitz'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Beelitz', 'FollowUp'] = 'Beelitz'

In [ ]:
All_MetaTables_ThisSub_Combined.head()

In [ ]:
All_MetaTables_ThisSub_Combined_FileName = f'{mySub}_MetaTable_All.json'

All_MetaTables_ThisSub_Combined.to_json(
    os.path.join(
        json_path_project, mySub, 
        All_MetaTables_ThisSub_Combined_FileName),
    orient ='records'
)

In [ ]:
sums = {}
# Get the unique time points in the order they appear
time_points = All_MetaTables_ThisSub_Combined['FollowUp'].unique()

# Sort the time points based on their first occurrence in the DataFrame
time_points_sorted = sorted(time_points, key=lambda x: np.where(All_MetaTables_ThisSub_Combined['FollowUp'] == x)[0][0])
# Get the corresponding y-values (last battery values)
last_battery_values = All_MetaTables_ThisSub_Combined.groupby('FollowUp')['BatteryPercentage'].last()
y_values = [last_battery_values[tp] if tp in last_battery_values.index else 0 for tp in time_points_sorted]


for tp in time_points_sorted:
    subset = All_MetaTables_ThisSub_Combined[All_MetaTables_ThisSub_Combined['FollowUp'] == tp]
    telemetry_sum = subset['telemetry_durationSec'].sum()
    sensing_sum = subset['overallSensingDurSec'].sum()
    last_battery_value = last_battery_values[tp] if tp in last_battery_values.index else None
    sums[tp] = {
        'Telemetry Duration Sum': telemetry_sum,
        'Sensing Duration Sum': sensing_sum,
        'Last Battery Value': last_battery_value
    }

# Print the sums dictionary
for tp, values in sums.items():
    print(f"FollowUp: {tp}, Sums: {values}")


In [ ]:
sums

In [ ]:
AvgValues_ThisSub_Combined_FileName = f'{mySub}_TotalValues.json'

with open(os.path.join(
        json_path_project, mySub, 
        AvgValues_ThisSub_Combined_FileName), 'w') as json_file:
            json.dump(sums, json_file)

In [ ]:
# Assuming you have the 'sums' dictionary from the previous code

# Extract the data for each subplot
last_battery_values = [sums[tp]['Last Battery Value'] for tp in time_points_sorted]
telemetry_sums = [sums[tp]['Telemetry Duration Sum'] for tp in time_points_sorted]
sensing_sums = [sums[tp]['Sensing Duration Sum'] for tp in time_points_sorted]

for i in range(len(telemetry_sums)):
    telemetry_sums[i] /= 60

for i in range(len(sensing_sums)):
    sensing_sums[i] /= 60

# Set up the subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# First subplot - 'Last Battery Value'
ax1.bar(time_points_sorted, last_battery_values)
ax1.set_ylabel('IPG Battery [%]')
ax1.set_ylim(60,100)

# Second subplot - Stacked bar plots
ind = np.arange(len(time_points_sorted))
width = 0.7
ax2.bar(ind, telemetry_sums, width)
ax2.bar(ind, sensing_sums, width, label='Sensing Duration Sum')
ax2.set_ylabel('Total Duration [min]')
ax2.set_xticks(ind)
ax2.set_xticklabels(time_points_sorted)
ax2.legend()

fig_title = f"{All_MetaTables_ThisSub_Combined['SubID'].unique()[0]}-Overview"
plt.suptitle(fig_title)
# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
AvgValues_ThisSub_FigName = f'{mySub}_Overview'

fig.savefig(os.path.join(
        json_path_project, mySub, AvgValues_ThisSub_FigName),
        dpi = 150)